# Create susceptibility charts

In this notebook we showcase how we can compute the multipactor thresholds measured during several tests and represent it on a susceptibility chart.

## Set up

Generic libraries:

In [1]:
from functools import partial
from pathlib import Path
import tomllib

import numpy as np

Other libraries required for this notebook:

In [2]:
from multipac_testbench.test_campaign import TestCampaign
import multipac_testbench.instruments as ins
from multipac_testbench.util.post_treaters import running_mean
from multipac_testbench.util.multipactor_detectors import \
    quantity_is_above_threshold

Define the project path, load the configuration.

In [3]:
project = Path("../data/campaign_one_probe/")
config_path = Path(project, "testbench_configuration.toml")

with open(config_path, "rb") as f:
    config = tomllib.load(f)

In [4]:
frequencies = (120., 160.)
swrs = (1., 1.)
filepaths = (
    Path(project, "120MHz_travelling.csv"),
    Path(project, "160MHz_travelling.csv"),
)
test_campaign = TestCampaign.from_filepaths(filepaths,a
                                            frequencies,
                                            swrs,
                                            config,
                                            sep='\t')

SyntaxError: invalid syntax. Perhaps you forgot a comma? (2176949027.py, line 7)

Prepare visualisation

In [ ]:
figsize = (8, 8)

## Calculate thresholds

Smooth the current data

In [ ]:
current_smoother = partial(
    running_mean,
    n_mean=10,
    mode='same',
)

test_campaign.add_post_treater(
    current_smoother,
    ins.CurrentProbe,
)

Set a multipactor detection criterion:

In [ ]:
current_multipactor_criterions = {'threshold': 16.,
                                  'consecutive_criterion': 10,
                                  'minimum_number_of_points': 5}
current_multipac_detector = partial(quantity_is_above_threshold,
                                    **current_multipactor_criterions)
power_growth_mask_kw = {'n_trailing_points_to_check': 70}
current_multipactor_bands = test_campaign.detect_multipactor(
    current_multipac_detector,
    ins.CurrentProbe,
    power_growth_mask_kw=power_growth_mask_kw,
)

<div class="alert alert-block alert-info"><b>Note:</b> Note that for one of the tests, we stopped the experimentation after the start of a new power growth. It messes with the thresholds, as we always take the most conditioned thresholds, *i.e.* the ones that are measured during the last power cycle. Hence we tell the script that the power does not grow again in the last 70 points.</div>

Check that detected multipactor zones are consistent:

## Create susceptibility chart

In [ ]:
axe, df_susceptibility = test_campaign.susceptibility(
    current_multipactor_bands,
    figsize=figsize,
    xlim=(80., 700.),
    ylim=(1e2, 1e4),
)

<div class="alert alert-block alert-info"><b>Note:</b> It would also work with a test with several electric field probes.</div>